### Setup

In [1]:
import numpy as np

from hot_dust import preprocess, model

In [2]:
from importlib import reload

preprocess = reload(preprocess)
model = reload(model)

In [3]:
ds = preprocess.prepare_training_data()
train, validate, test = preprocess.to_tensorflow(preprocess.split_training_data(ds))

### Parameters

In [4]:
epochs = 2000
batch_size = 64
buffer_size = 10 * batch_size

model = reload(model)  # DEBUG

train = train.shuffle(buffer_size).batch(batch_size)
validate = validate.batch(batch_size)

layer = model.pretraining(dataset=test)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [5]:
network = model.compile(normalization=layer)
network.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 11)]              0         
                                                                 
 normalization (Normalizatio  (None, 11)               23        
 n)                                                              
                                                                 
 dense (Dense)               (None, 32)                384       
                                                                 
 dense_3 (Dense)             (None, 8)                 264       
                                                                 
 dense_4 (Dense)             (None, 1)                 9         
                                                                 
Total params: 680
Trainable params: 657
Non-trainable params: 23
______________________________________________________________

### Fitting

In [6]:
fit = network.fit(
    train,
    epochs=epochs,
    validation_data=validate,
    verbose=2,
)
network.save("data/network")
fit = {"epoch": fit.epoch, **fit.history}
np.savez("data/fit.npz", fit)

Epoch 1/2000
126/126 - 1s - loss: 5.9898 - val_loss: 5.2647 - 718ms/epoch - 6ms/step
Epoch 2/2000
126/126 - 0s - loss: 4.7515 - val_loss: 4.5712 - 142ms/epoch - 1ms/step
Epoch 3/2000
126/126 - 0s - loss: 4.2577 - val_loss: 4.1351 - 112ms/epoch - 886us/step
Epoch 4/2000
126/126 - 0s - loss: 3.8982 - val_loss: 3.8054 - 112ms/epoch - 887us/step
Epoch 5/2000
126/126 - 0s - loss: 3.6098 - val_loss: 3.5248 - 105ms/epoch - 835us/step
Epoch 6/2000
126/126 - 0s - loss: 3.3580 - val_loss: 3.2831 - 112ms/epoch - 889us/step
Epoch 7/2000
126/126 - 0s - loss: 3.1413 - val_loss: 3.0749 - 111ms/epoch - 882us/step
Epoch 8/2000
126/126 - 0s - loss: 2.9563 - val_loss: 2.9025 - 107ms/epoch - 849us/step
Epoch 9/2000
126/126 - 0s - loss: 2.8013 - val_loss: 2.7577 - 221ms/epoch - 2ms/step
Epoch 10/2000
126/126 - 0s - loss: 2.6694 - val_loss: 2.6317 - 108ms/epoch - 853us/step
Epoch 11/2000
126/126 - 0s - loss: 2.5586 - val_loss: 2.5314 - 170ms/epoch - 1ms/step
Epoch 12/2000
126/126 - 0s - loss: 2.4645 - val_l

INFO:tensorflow:Assets written to: data/network\assets


INFO:tensorflow:Assets written to: data/network\assets


### Training and Validation Losses

In [7]:
model.plot_loss(fit)

:NdOverlay   [Variable]
   :Scatter   [epoch]   (value)